# From raw data to processed data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

## Import data

### All measured data

In [3]:
#All uncorrected data
cps_raw_pivot = ((pd.read_csv('data/cps_raw_pivot2.csv', sep = ',')).pivot_table(index=["Polymer", "Time", "Source","Batch","Color", "Day","Day2", "N"])
                     .drop(columns=['Cl37', 'Cu65', 'Pb206', 'Pb207', 'Zr90','Cr50'])
                     .drop(columns=['index'])
                     .drop(columns=['Co59', 'Pd106']) #Removed because those columns have low values in all objects
                     .drop(columns=['Ga69']) #Removed because of high correlation
                    )

cps_raw_pivot = cps_raw_pivot[sorted(cps_raw_pivot.columns, key=lambda x: int(re.search(r'\d+', x).group()))] # Sort columns on atomic mass

cps_raw_pivot

C13  \
Polymer Time    Source Batch Color  Day    Day2 N                 
Can     Between 01     A     Black  230605 1    1  1.478820e+06   
                                                2  1.319522e+06   
                                                3  1.183784e+06   
                                                4  1.124131e+06   
                                    230627 2    1  1.687216e+06   
...                                                         ...   
Wire    Between 49     A     Yellow 230601 1    4  5.068127e+05   
                                    230623 2    1  6.772993e+05   
                                                2  7.156011e+05   
                                                3  7.147296e+05   
                                                4  6.430360e+05   

                                                            Na23  \
Polymer Time    Source Batch Color  Day    Day2 N                  
Can     Between 01     A     Black  230605 1    1   56841.704042   
                                                2   48238.705174   
                                                3   66655.715274   
                                                4   42589.440131   
                                    230627 2    1   59406.208408   
...                                                          ...   
Wire    Between 49     A     Yellow 230601 1    4  241226.265339   
                                    230623 2    1  272082.121049   
                                                2  350748.483015   
                                                3  267475.081084   
                                                4  225408.827133   

                                                            Mg25  \
Polymer Time    Source Batch Color  Day    Day2 N                  
Can     Between 01     A     Black  230605 1    1   12105.466885   
                                                2   12876.598394   
                                                3   10404.698679   
                                                4    9366.842277   
                                    230627 2    1   13120.011733   
...                                                          ...   
Wire    Between 49     A     Yellow 230601 1    4  666638.362292   
                                    230623 2    1  750185.849410   
                                                2  766291.161143   
                                                3  733973.700897   
                                                4  676966.875409   

                                                           Al27          Si29  \
Polymer Time    Source Batch Color  Day    Day2 N                               
Can     Between 01     A     Black  230605 1    1  4.514513e+05  38445.552027   
                                                2  2.649830e+05  37575.190459   
                                                3  3.659630e+05  38472.150295   
                                                4  2.403186e+05  33712.701265   
                                    230627 2    1  4.742555e+05  41617.892665   
...                                                         ...           ...   
Wire    Between 49     A     Yellow 230601 1    4  6.260520e+05  25576.589605   
                                    230623 2    1  7.211286e+05  28154.509971   
                                                2  1.268050e+06  58711.798741   
                                                3  6.270178e+05  32438.365223   
                                                4  6.816793e+05  29630.050526   

                                                            K39          Ca44  \
Polymer Time    Source Batch Color  Day    Day2 N                               
Can     Between 01     A     Black  230605 1    1  2.673996e+05  1.156666e+06   
                                                2  1.375284e+05  1.192630e+06   
                                        

### Concentrations standard

In [4]:
conc_all_standards = ((pd.read_csv('data/Standard_concentrations.csv', sep = ';')).pivot_table(index=["Polymer", "Conc (mg/kg)"])
                      .drop(columns=['Zr90','Co59', 'Pd105','Cr53','Ga71'])
                     )

In [5]:
conc_standards = conc_all_standards.xs(level="Conc (mg/kg)", key="High")
conc_standards

,Al27,As75,Ba137,Ca44,Cu63,Fe56,K39,Mg24,Mn55,Na23,Nb93,Ni60,Pb208,Sb121,Si28,Sn118,Sr88,Ti47
Polymer,,,,,,,,,,,,,,,,,,
PE,1000,200.0,1000,2000,200,200,200,1000,280.0,200,200,100.0,200,1000,2000,200,400,2000
PS,500,50.0,500,1000,100,500,100,500,140.0,100,100,50.0,100,500,1000,100,100,500
PVC,500,500.0,500,1000,500,200,100,500,140.0,500,100,20.0,500,1000,1000,200,100,1000


## Datasets

The data of the forensic objects (FO) is normalized and quantified in various ways.

In [6]:
data_FO_raw = (cps_raw_pivot
               .reset_index()
               .loc[cps_raw_pivot.reset_index()['Polymer'].str.contains('Can|Wire|Tape|Tubing')]
               .pivot_table(index=['Polymer', 'Source', 'Time', 'Day','Day2', 'Batch', "Color", 'N'])
              )

### Data Forensic Objects (data_FO)

In [8]:
data_FO = data_FO_raw.drop(columns=['C13'])
data_FO

Al27         As75  \
Polymer Source Time    Day    Day2 Batch Color  N                              
Can     01     Between 230605 1    A     Black  1  4.514513e+05  2424.022697   
                                                2  2.649830e+05  2010.655483   
                                                3  3.659630e+05  1982.227454   
                                                4  2.403186e+05  2078.278705   
                       230627 2    A     Black  1  4.742555e+05   454.148398   
...                                                         ...          ...   
Wire    49     Between 230601 1    A     Yellow 4  6.260520e+05  2449.337480   
                       230623 2    A     Yellow 1  7.211286e+05  1918.783449   
                                                2  1.268050e+06  2072.578929   
                                                3  6.270178e+05  1772.158243   
                                                4  6.816793e+05  1463.358616   

                                                           Ba138  \
Polymer Source Time    Day    Day2 Batch Color  N                  
Can     01     Between 230605 1    A     Black  1   86832.257747   
                                                2  113244.518728   
                                                3  126763.638529   
                                                4   85428.380212   
                       230627 2    A     Black  1   74589.797046   
...                                                          ...   
Wire    49     Between 230601 1    A     Yellow 4   38155.721808   
                       230623 2    A     Yellow 1   52927.602727   
                                                2   83856.410263   
                                                3   40769.225489   
                                                4   37683.452751   

                                                           Ca44  \
Polymer Source Time    Day    Day2 Batch Color  N                 
Can     01     Between 230605 1    A     Black  1  1.156666e+06   
                                                2  1.192630e+06   
                                                3  1.179015e+06   
                                                4  1.205820e+06   
                       230627 2    A     Black  1  9.902725e+05   
...                                                         ...   
Wire    49     Between 230601 1    A     Yellow 4  4.420022e+07   
                       230623 2    A     Yellow 1  4.210882e+07   
                                                2  4.983738e+07   
                                                3  4.757561e+07   
                                                4  4.503496e+07   

                                                            Cu63  \
Polymer Source Time    Day    Day2 Batch Color  N                  
Can     01     Between 230605 1    A     Black  1   98427.319507   
                                                2  101432.243981   
                                                3  100321.324477   
                                                4   97302.642666   
                       230627 2    A     Black  1   89048.742400   
...                                                          ...   
Wire    49     Between 230601 1    A     Yellow 4   14561.255521   
                       230623 2    A     Yellow 1   14997.490717   
                                                2   19201.416678   
                                                3   22478.465137   
                                                4   12912.982289   

                                                           Fe57           K39  \
Polymer Source Time    Day    Day2 Batch Color  N                               
Can     01     Between 230605 1    A     Black  1  36682.518303  2.673996e+05   
                                                2  32799.164407  1.375284e+05   
                                                3  4

In [9]:
data_FO_mean = data_FO.groupby(['Polymer', 'Source','Color']).agg('mean')
data_FO_mean

Al27         As75         Ba138          Ca44  \
Polymer Source Color                                                           
Can     01     Black   3.965128e+05  1265.862623  1.034554e+05  1.102802e+06   
        02     Black   1.162993e+05  1186.196410  2.443738e+04  4.566638e+05   
        03     Yellow  3.144475e+04  2043.685855  1.047676e+04  5.331150e+04   
        04     Red     1.345701e+05  4449.362376  4.368633e+03  4.222358e+05   
        05     Black   5.862924e+04  1523.091000  8.049244e+03  3.165951e+04   
...                             ...          ...           ...           ...   
Wire    45     Yellow  1.697523e+06  3618.799215  1.094877e+05  6.963226e+07   
        46     Yellow  1.070208e+08  2277.025631  1.570979e+06  3.140630e+07   
        47     Yellow  2.010455e+07  1972.930057  1.125623e+05  4.368050e+07   
        48     Yellow  1.810579e+07  2113.736778  5.901743e+04  3.690277e+07   
        49     Yellow  7.070563e+05  2693.880520  2.602684e+05  4.588283e+07   

                                Cu63           Fe57           K39  \
Polymer Source Color                                                
Can     01     Black    97286.743636   40389.034592  1.880383e+05   
        02     Black     1202.002196    3511.525340  1.351098e+05   
        03     Yellow    3738.831772     680.375746  4.059146e+04   
        04     Red       1336.330290  424033.741735  5.599446e+04   
        05     Black     2492.447846   18183.805944  1.817360e+04   
...                              ...            ...           ...   
Wire    45     Yellow  240729.411142  137962.920089  1.392728e+06   
        46     Yellow   61770.005384  119087.278902  1.717437e+06   
        47     Yellow   28946.853391  113591.148982  1.482310e+06   
        48     Yellow   58338.036698   69569.977191  8.588230e+05   
        49     Yellow   18564.598139   83100.246937  5.524688e+05   

                               Mg25          Mn55          Na23  \
Polymer Source Color                                              
Can     01     Black   1.086084e+04  3.895125e+04  6.184298e+04   
        02     Black   3.923526e+03  1.823699e+04  2.547112e+04   
        03     Yellow  1.572615e+03  1.844083e+03  4.781469e+04   
        04     Red     2.707714e+05  1.157116e+04  2.201130e+05   
        05     Black   1.498524e+03  1.168083e+03  2.721214e+04   
...                             ...           ...           ...   
Wire    45     Yellow  5.712382e+05  1.184430e+06  9.142881e+05   
        46     Yellow  7.024733e+06  9.240752e+05  2.064116e+06   
        47     Yellow  3.387040e+06  1.355134e+06  6.353462e+05   
        48     Yellow  3.524418e+06  6.584345e+05  5.007152e+05   
        49     Yellow  6.982908e+05  1.375710e+05  2.758768e+05   

                                Nb93          Ni60         Pb208  \
Polymer Source Color                                               
Can     01     Black    11745.415866    870.861835   2117.389283   
        02     Black      243.059593    262.686565   3362.444678   
        03     Yellow    6983.995041    198.218505    733.778183   
        04     Red      21917.095634    490.721551   1578.208300   
        05     Black      448.487312    167.932389    459.107674   
...                              ...           ...           ...   
Wire    45     Yellow     525.442076  11218.504724  12352.336115   
        46     Yellow  108011.078889   9098.817793  34483.146061   
        47     Yellow     905.157758  11361.398139  10442.587911   
        48     Yellow     514.196845   4797.510239   6169.996874   
        49     Yellow     681.071469   3243.976918  12948.382160   

                               Sb121          Si29          Sn118  \
Polymer Source Color                                                
Can     01     Black    30381.952941  3.874212e+04     975.097790   
        02     Black    25914.888073  3.497713e+04    2366.505480   
        03     Yellow   13043.403107  2.069595e+04     888.

### Normalization by element X (data_FO_X)

Optionally, C13 can be changed to another element.

In [11]:
data_FO_X = data_FO_raw.divide((data_FO_raw['C13']).values, axis=0).drop(columns=['C13'])
data_FO_X

Al27      As75  \
Polymer Source Time    Day    Day2 Batch Color  N                       
Can     01     Between 230605 1    A     Black  1  0.305278  0.001639   
                                                2  0.200817  0.001524   
                                                3  0.309147  0.001674   
                                                4  0.213782  0.001849   
                       230627 2    A     Black  1  0.281088  0.000269   
...                                                     ...       ...   
Wire    49     Between 230601 1    A     Yellow 4  1.235273  0.004833   
                       230623 2    A     Yellow 1  1.064712  0.002833   
                                                2  1.772008  0.002896   
                                                3  0.877280  0.002479   
                                                4  1.060095  0.002276   

                                                      Ba138       Ca44  \
Polymer Source Time    Day    Day2 Batch Color  N                        
Can     01     Between 230605 1    A     Black  1  0.058717   0.782155   
                                                2  0.085822   0.903834   
                                                3  0.107083   0.995971   
                                                4  0.075995   1.072668   
                       230627 2    A     Black  1  0.044209   0.586927   
...                                                     ...        ...   
Wire    49     Between 230601 1    A     Yellow 4  0.075286  87.212130   
                       230623 2    A     Yellow 1  0.078145  62.171658   
                                                2  0.117183  69.644090   
                                                3  0.057041  66.564477   
                                                4  0.058602  70.034890   

                                                       Cu63      Fe57  \
Polymer Source Time    Day    Day2 Batch Color  N                       
Can     01     Between 230605 1    A     Black  1  0.066558  0.024805   
                                                2  0.076870  0.024857   
                                                3  0.084746  0.038417   
                                                4  0.086558  0.039470   
                       230627 2    A     Black  1  0.052779  0.025979   
...                                                     ...       ...   
Wire    49     Between 230601 1    A     Yellow 4  0.028731  0.159453   
                       230623 2    A     Yellow 1  0.022143  0.113833   
                                                2  0.026833  0.122830   
                                                3  0.031450  0.122148   
                                                4  0.020081  0.126010   

                                                        K39      Mg25  \
Polymer Source Time    Day    Day2 Batch Color  N                       
Can     01     Between 230605 1    A     Black  1  0.180820  0.008186   
                                                2  0.104226  0.009759   
                                                3  0.148206  0.008789   
                                                4  0.092363  0.008333   
                       230627 2    A     Black  1  0.125188  0.007776   
...                                                     ...       ...   
Wire    49     Between 230601 1    A     Yellow 4  0.884848  1.315354   
                       230623 2    A     Yellow 1  0.847742  1.107613   
                                                2  1.466826  1.070836   
                                                3  0.661975  1.026925   
                                                4  0.653773  1.052767   

                                                       Mn55      Na23  \
Polymer Source Time    Day    Day2 Batch Color  N                       
Can     01     Between 230605 1    A     Black  1  0.029809  0.038437   
                       

In [12]:
data_FO_mean_X = data_FO_X.groupby(['Polymer', 'Source','Color']).agg('mean')

### Correction for glass standard (data_FO_SRM)

In [13]:
data_SRM = (cps_raw_pivot
               .reset_index()
               .loc[cps_raw_pivot.reset_index()['Polymer'].str.contains('Glass')]
               .pivot_table(index=['Polymer', 'Source', 'Time', 'Day','Day2', 'Batch', "Color", 'N'])
              )

In [14]:
data_SRM_mean = (data_SRM
                    .reset_index() 
                    .groupby(['Polymer', 'Source', 'Time', 'Day','Day2', 'Batch', "Color"])                    
                    .agg('mean')
                    .drop(columns=['N'])
                    .pivot_table(index=['Day'])
                )

In [15]:
data_cps_SRM = data_FO_raw.div(data_SRM_mean,axis=0).dropna().drop(columns=['C13'])

In [16]:
#The corrected cps are calculated. When the concentration is known, the cps can be converted to masses (for most elements this is available)
#data_FO_SRM = (data_cps_SRM*conc_SRM.values)
data_FO_SRM = data_cps_SRM

In [17]:
data_FO_mean_SRM = data_FO_SRM.groupby(['Polymer', 'Source','Color']).agg('mean')

### Quantification with PVC standard (data_FO_PVC)

In [18]:
data_PVC = (cps_raw_pivot
               .reset_index()
               .loc[cps_raw_pivot.reset_index()['Polymer'].str.contains('PVC')]
               .pivot_table(index=['Polymer', 'Source', 'Time', 'Day', 'Day2', 'Batch', "Color", 'N'])
              )

In [19]:
data_PVC_mean = (data_PVC
                     .reset_index()
                     .drop(columns=['Time', 'Batch', 'N'])
                     .groupby(['Polymer', 'Source', 'Day','Day2',"Color"])
                     .agg('mean')
                     .pivot_table(index=['Day'])
                )

In [20]:
data_cps_PVC = data_FO_raw.div(data_PVC_mean,axis=0).dropna().drop(columns=['C13'])
data_cps_PVC

Al27      As75  \
Polymer Source Time    Day    Day2 Batch Color  N                       
Can     01     Between 230605 1    A     Black  1  0.210409  0.007316   
                                                2  0.123501  0.006069   
                                                3  0.170565  0.005983   
                                                4  0.112006  0.006273   
                       230627 2    A     Black  1  0.081064  0.000534   
...                                                     ...       ...   
Wire    49     Between 230601 1    A     Yellow 4  0.331431  0.007389   
                       230623 2    A     Yellow 1  0.092688  0.001846   
                                                2  0.162985  0.001994   
                                                3  0.080592  0.001705   
                                                4  0.087618  0.001408   

                                                      Ba138        Ca44  \
Polymer Source Time    Day    Day2 Batch Color  N                         
Can     01     Between 230605 1    A     Black  1  0.010282    3.661105   
                                                2  0.013410    3.774938   
                                                3  0.015011    3.731843   
                                                4  0.010116    3.816689   
                       230627 2    A     Black  1  0.003926    1.881315   
...                                                     ...         ...   
Wire    49     Between 230601 1    A     Yellow 4  0.004929  138.363967   
                       230623 2    A     Yellow 1  0.002388   67.401159   
                                                2  0.003784   79.771822   
                                                3  0.001840   76.151524   
                                                4  0.001700   72.084853   

                                                       Cu63      Fe57  \
Polymer Source Time    Day    Day2 Batch Color  N                       
Can     01     Between 230605 1    A     Black  1  0.066414  0.866988   
                                                2  0.068442  0.775206   
                                                3  0.067692  1.074851   
                                                4  0.065656  1.048668   
                       230627 2    A     Black  1  0.016849  0.528409   
...                                                     ...       ...   
Wire    49     Between 230601 1    A     Yellow 4  0.010083  1.899770   
                       230623 2    A     Yellow 1  0.002374  0.780766   
                                                2  0.003040  0.890116   
                                                3  0.003559  0.884097   
                                                4  0.002044  0.820564   

                                                        K39      Mg25  \
Polymer Source Time    Day    Day2 Batch Color  N                       
Can     01     Between 230605 1    A     Black  1  0.173529  0.090403   
                                                2  0.089249  0.096161   
                                                3  0.113854  0.077701   
                                                4  0.067379  0.069951   
                       230627 2    A     Black  1  0.061287  0.061783   
...                                                     ...       ...   
Wire    49     Between 230601 1    A     Yellow 4  0.288898  5.013988   
                       230623 2    A     Yellow 1  0.148482  3.002561   
                                                2  0.271442  3.067022   
                                                3  0.122352  2.937673   
                                                4  0.108715  2.709508   

                                                       Mn55      Na23  \
Polymer Source Time    Day    Day2 Batch Color  N                       
Can     01     Between 230605 1    A     Black  1  0.009719  0.021823   
          

In [21]:
conc_PVC = pd.DataFrame(conc_standards.loc["PVC"]).transpose()

In [22]:
data_FO_PVC = data_cps_PVC*conc_PVC.values

In [23]:
data_FO_mean_PVC = data_FO_PVC.groupby(['Polymer', 'Source','Color']).agg('mean')

### Quantification with PE standard (data_FO_PE)

In [24]:
data_PE = (cps_raw_pivot
               .reset_index()
               .loc[cps_raw_pivot.reset_index()['Polymer'].str.contains('PE')]
               .pivot_table(index=['Polymer', 'Source', 'Time', 'Day', 'Day2','Batch', "Color", 'N'])
              )

In [25]:
data_PE_mean = (data_PE
                     .reset_index()
                     .drop(columns=['Time', 'Batch', 'N'])
                     .groupby(['Polymer', 'Source', 'Day','Day2', "Color"])
                     .agg('mean')
                     .pivot_table(index=['Day'])
                )

In [26]:
data_cps_PE = data_FO_raw.div(data_PE_mean,axis=0).dropna().drop(columns=['C13'])

In [27]:
conc_PE = pd.DataFrame(conc_standards.loc["PE"]).transpose()

In [28]:
data_FO_PE = data_cps_PE*conc_PE.values

In [29]:
data_FO_mean_PE = data_FO_PE.groupby(['Polymer', 'Source','Color']).agg('mean')

### Quantification with PS standard (data_FO_PS)

In [30]:
data_PS = (cps_raw_pivot
               .reset_index()
               .loc[cps_raw_pivot.reset_index()['Polymer'].str.contains('PS')]
               .pivot_table(index=['Polymer', 'Source', 'Time', 'Day','Day2', 'Batch', "Color", 'N'])
              )

In [31]:
data_PS_mean = (data_PS
                     .reset_index()
                     .drop(columns=['Time', 'Batch', 'N'])
                     .groupby(['Polymer', 'Source', 'Day', 'Day2',"Color"])
                     .agg('mean')
                     .pivot_table(index=['Day'])
                )

In [32]:
data_cps_PS = data_FO_raw.div(data_PS_mean,axis=0).dropna().drop(columns=['C13'])

In [33]:
conc_PS = pd.DataFrame(conc_standards.loc["PS"]).transpose()

In [34]:
data_FO_PS = data_cps_PS*conc_PS.values

In [35]:
data_FO_mean_PS = data_FO_PS.groupby(['Polymer', 'Source','Color']).agg('mean')

### Correction for norm glass (data_FO_SRM_X)

Optionally, C13 can be changed to another element.

In [37]:
data_SRM_X = data_SRM.divide(data_SRM['C13'].values, axis=0).drop(columns=['C13']) #.drop(columns=['Ca44','C13']) #

In [38]:
data_SRM_mean_X = (data_SRM_X
                     .reset_index()
                     .drop(columns=['Time', 'Batch', 'N'])
                     .groupby(['Polymer', 'Source', 'Day','Day2', "Color"])
                     .agg('mean')
                     .pivot_table(index=['Day'])
                )

In [39]:
data_cps_SRM_X = data_FO_X.div(data_SRM_mean_X,axis=0).dropna()

In [40]:
#This step can be performed if conc. SRM is known
#data_FO_SRM_X = (data_cps_SRM_X*conc_SRM.values)
data_FO_SRM_X = data_cps_SRM_X

In [41]:
data_FO_mean_SRM_X = data_cps_SRM_X.groupby(['Polymer', 'Source','Color']).agg('mean')

### Quantification norm PVC (data_FO_PVC_X)

Optionally, C13 can be changed to another element.

In [43]:
data_PVC_X = data_PVC.divide(data_PVC['C13'].values, axis=0).drop(columns=['C13']) #.drop(columns=['Ca44','C13']) #

In [44]:
data_PVC_mean_X = (data_PVC_X
                     .reset_index()
                     .drop(columns=['Time', 'Batch', 'N'])
                     .groupby(['Polymer', 'Source', 'Day','Day2', "Color"])
                     .agg('mean')
                     .pivot_table(index=['Day'])
                )

In [45]:
data_cps_PVC_X = data_FO_X.div(data_PVC_mean_X,axis=0).dropna()

In [46]:
conc_PVC_corr = conc_PVC #.reset_index().drop(columns=['Ca44']).set_index('index')

In [47]:
data_FO_PVC_X = (data_cps_PVC_X*conc_PVC_corr.values)

In [48]:
data_FO_mean_PVC_X = data_FO_PVC_X.groupby(['Polymer', 'Source','Color']).agg('mean')

### Quantification norm PE (data_FO_PE_X)

Optionally, C13 can be changed to another element.

In [50]:
data_PE_X = data_PE.divide(data_PE['C13'].values, axis=0).drop(columns=['C13']) #.drop(columns=['Ca44','C13']) #

In [51]:
data_PE_mean_X = (data_PE_X
                     .reset_index()
                     .drop(columns=['Time', 'Batch', 'N'])
                     .groupby(['Polymer', 'Source', 'Day','Day2', "Color"])
                     .agg('mean')
                     .pivot_table(index=['Day'])
                )

In [52]:
data_cps_PE_X = data_FO_X.div(data_PE_mean_X,axis=0).dropna()

In [53]:
conc_PE_corr = conc_PE #.reset_index().drop(columns=['Ca44']).set_index('index')

In [54]:
data_FO_PE_X = (data_cps_PE_X*conc_PE_corr.values)

In [55]:
data_FO_mean_PE_X = data_FO_PE_X.groupby(['Polymer', 'Source','Color']).agg('mean')

### Quantification norm PS (data_FO_PS_X)

Optionally, C13 can be changed to another element.

In [57]:
data_PS_X = data_PS.divide(data_PS['C13'].values, axis=0).drop(columns=['C13']) #.drop(columns=['Ca44','C13']) #

In [58]:
data_PS_mean_X = (data_PS_X
                     .reset_index()
                     .drop(columns=['Time', 'Batch', 'N'])
                     .groupby(['Polymer', 'Source', 'Day', 'Day2',"Color"])
                     .agg('mean')
                     .pivot_table(index=['Day'])
                )

In [59]:
data_cps_PS_X = data_FO_X.div(data_PS_mean_X,axis=0).dropna()

In [60]:
conc_PS_corr = conc_PS #.reset_index().drop(columns=['Ca44']).set_index('index')

In [61]:
data_FO_PS_X = (data_cps_PS_X*conc_PS_corr.values)

In [62]:
data_FO_mean_PS_X = data_FO_PS_X.groupby(['Polymer', 'Source','Color']).agg('mean')

## Final dataset

### Save dataset

Select and save the csv file with the data which you want to process further. The following options are possible:
* data_FO (uncorrected data)
* data_FO_X (data normalized to C, Al, or Ca signal)
* Data_FO_SRM (data (semi-)quantified with NIST SRM 612 standard, raw signal divided by signal standard)
* Data_FO_PVC (data quantified with PVC standard in mg/kg)
* Data_FO_PE (data quantified with PE standard in mg/kg)
* Data_FO_PS (data quantified with PS standard in mg/kg)
* Data_FO_SRM_X (data normalized to C, Al, or Ca signal and (semi-)quantified with NIST SRM 612 standard)
* Data_FO_PVC_X (data normalized to C, Al, or Ca signal and (semi-)quantified with PVC standard in mg/kg)
* Data_FO_PE_X (data normalized to C, Al, or Ca signal and (semi-)quantified with PE standard in mg/kg)
* Data_FO_PS_X (data normalized to C, Al, or Ca signal and (semi-)quantified with PS standard in mg/kg)

In [63]:
# Select dataset with specific normalization and quantification method
data_FO_select = data_FO

data_x = (data_FO_select.reset_index()
             .drop(columns=['Time','Day'])
             .replace(0.000000, 0.000001)
             .pivot_table(index=['Polymer', 'Source','Day2', 'Batch', "Color", 'N'])
             .sort_values(["Polymer","Source","Batch","Day2","N"])
         )[sorted(data_FO_select.columns, key=lambda x: int(re.search(r'\d+', x).group()))]

#data_x.to_csv('df/Data_Polymers.csv') 

data_x

Na23           Mg25  \
Polymer Source Day2 Batch Color  N                                 
Can     01     1    A     Black  1   56841.704042   12105.466885   
                                 2   48238.705174   12876.598394   
                                 3   66655.715274   10404.698679   
                                 4   42589.440131    9366.842277   
               2    A     Black  1   59406.208408   13120.011733   
...                                           ...            ...   
Wire    49     1    A     Yellow 4  241226.265339  666638.362292   
               2    A     Yellow 1  272082.121049  750185.849410   
                                 2  350748.483015  766291.161143   
                                 3  267475.081084  733973.700897   
                                 4  225408.827133  676966.875409   

                                            Al27          Si29           K39  \
Polymer Source Day2 Batch Color  N                                             
Can     01     1    A     Black  1  4.514513e+05  38445.552027  2.673996e+05   
                                 2  2.649830e+05  37575.190459  1.375284e+05   
                                 3  3.659630e+05  38472.150295  1.754441e+05   
                                 4  2.403186e+05  33712.701265  1.038278e+05   
               2    A     Black  1  4.742555e+05  41617.892665  2.112191e+05   
...                                          ...           ...           ...   
Wire    49     1    A     Yellow 4  6.260520e+05  25576.589605  4.484523e+05   
               2    A     Yellow 1  7.211286e+05  28154.509971  5.741748e+05   
                                 2  1.268050e+06  58711.798741  1.049662e+06   
                                 3  6.270178e+05  32438.365223  4.731333e+05   
                                 4  6.816793e+05  29630.050526  4.203999e+05   

                                            Ca44           Ti47  \
Polymer Source Day2 Batch Color  N                                
Can     01     1    A     Black  1  1.156666e+06  500023.593316   
                                 2  1.192630e+06  503009.895749   
                                 3  1.179015e+06  531205.667919   
                                 4  1.205820e+06  513990.273923   
               2    A     Black  1  9.902725e+05  578351.405137   
...                                          ...            ...   
Wire    49     1    A     Yellow 4  4.420022e+07  303471.897709   
               2    A     Yellow 1  4.210882e+07  219515.724799   
                                 2  4.983738e+07  247101.927386   
                                 3  4.757561e+07  260551.362839   
                                 4  4.503496e+07  271233.637172   

                                             Mn55          Fe57         Ni60  \
Polymer Source Day2 Batch Color  N                                             
Can     01     1    A     Black  1   44082.572733  36682.518303   938.330928   
                                 2   39485.795384  32799.164407   799.352018   
                                 3   53590.660179  45477.240574   833.043514   
                                 4   55179.628033  44369.432417   821.664077   
               2    A     Black  1   34579.652692  43832.276618   905.254527   
...                                           ...           ...          ...   
Wire    49     1    A     Yellow 4  137095.732325  80812.833536  3105.095140   
               2    A     Yellow 1  123392.992913  77099.098742  2628.620746   
                                 2  140573.952415  87897.146067  3141.728652   
                                 3  155588.712507  87302.818301  2793.010369   
                                 4  123298.645865  81029.085051  2790.674752   

                                             Cu63         As75          Sr88  \
Polymer Source Day2 Batch Color  N                                             
Can     01     1    A     Black  1   98427.319